In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StringType
from pyspark.sql import functions as F

# Khởi tạo SparkSession
spark = SparkSession.builder.master("local").appName("Skills Standard").getOrCreate()

# Đọc dữ liệu từ file CSV
file1 = spark.read.option("header", "true").csv("./skills.csv")
file2 = spark.read.option("header", "true").csv("./combined_output_no_url.csv")

# Chuẩn bị file1 với cột skill và standard
file1 = file1.select("skill", "standard")

# Chuyển đổi skill thành dạng bảng với mỗi skill và standard
file1 = file1.withColumn("skill", F.lower(col("skill")))

# Broadcast file1 (để tránh lỗi về SparkContext)
broadcast_file1 = spark.sparkContext.broadcast(file1.collect())

# Hàm tạo ra trường skills_standard
def add_skills_standard(job_skills):
    skills_list = job_skills.split(",")
    standard_list = []
    
    # Duyệt qua từng kỹ năng và tìm kiếm trong broadcast_file1
    for skill in skills_list:
        skill = skill.strip().lower()
        
        # Tìm kiếm standard cho skill trong broadcast data
        for row in broadcast_file1.value:
            if row['skill'] == skill:
                standard_list.append(row['standard'])
                break
    
    return ",".join(standard_list)

# Đăng ký hàm UDF (User Defined Function)
add_skills_standard_udf = F.udf(add_skills_standard, StringType())

# Thêm cột skills_standard vào file2
file2 = file2.withColumn("skills_standard", add_skills_standard_udf(col("job_skills")))

# Hiển thị kết quả
file2.show(truncate=False)


+---------------------------------------------------------------------------------------------+----------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------+--------------+----------------------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
# Hàm tạo ra trường skills_standard với kiểm tra None hoặc rỗng
def add_skills_standard(job_skills):
    if job_skills is None or job_skills.strip() == "":  # Kiểm tra nếu job_skills là None hoặc rỗng
        return ""
    
    skills_list = job_skills.split(",")
    standard_list = []
    
    # Duyệt qua từng kỹ năng và tìm kiếm trong broadcast_file1
    for skill in skills_list:
        skill = skill.strip().lower()
        
        # Tìm kiếm standard cho skill trong broadcast data
        for row in broadcast_file1.value:
            if row['skill'] == skill:
                standard_list.append(row['standard'])
                break
    
    return ",".join(standard_list)

# Đăng ký hàm UDF (User Defined Function)
add_skills_standard_udf = F.udf(add_skills_standard, StringType())

# Thêm cột skills_standard vào file2
file2 = file2.withColumn("skills_standard", add_skills_standard_udf(col("job_skills")))

# Đếm số dòng có trường skills_standard rỗng hoặc null
count_null_or_empty = file2.filter((col("skills_standard").isNull()) | (col("skills_standard") == "")).count()

# Đếm tổng số dòng
total_rows = file2.count()

# Tính tỷ lệ phần trăm số dòng có skills_standard rỗng
percentage_empty = (count_null_or_empty / total_rows) * 100

# In kết quả
print(f"Số dòng có skills_standard rỗng: {count_null_or_empty}")
print(f"Tổng số dòng: {total_rows}")
print(f"Tỷ lệ phần trăm dòng có skills_standard rỗng: {percentage_empty:.2f}%")


Số dòng có skills_standard rỗng: 42398
Tổng số dòng: 114462
Tỷ lệ phần trăm dòng có skills_standard rỗng: 37.04%
